In [5]:
import tensorflow as tf 
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.layers import GlobalAveragePooling2D , Dense , Lambda ,Input
from tensorflow.keras.models import  Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.resnet50 import preprocess_input  
import numpy as np 
import os 

In [6]:
base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(224,224,3))


In [7]:
def create_feature():
    input_layer = Input(shape=(224,224,3))
    x = base_model(input_layer)
    x= GlobalAveragePooling2D()(x)
    return Model(input_layer,x)

feature_extractor = create_feature()   

In [12]:
def build_siamese_network():
    input_1 = Input(shape=(224,224,3))
    input_2 = Input(shape=(224,224,3))
    
    feature_1 = feature_extractor(input_1)
    feature_2 = feature_extractor(input_2)

    l2_distance = Lambda(lambda tensors : tf.keras.backend.abs(tensors[0]-tensors[1]))([feature_1,feature_2])
    x = Dense(512,activation='relu')(l2_distance)
    x = Dense(128,activation='relu')(x)

    output = Dense(3,activation='softmax')(x)

    siamese_model = Model(inputs=[input_1,input_2],outputs=output)

    siamese_model.compile(optimizer='adam',loss= 'categorical_crossentropy' ,metrics=['accuracy'])
    
    return siamese_model


In [14]:
siamese_model =  build_siamese_network()
siamese_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 2048)      │ 23,587,712 │ input_layer_9[0]… │
│ (Functional)        │                   │            │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 2048)      │          0 │ functional_1[6][… │
│                     │                   │            │ functional_1[7][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 512)       │  1,049,088 │ lambda_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │     65,664 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 3)         │        387 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,702,851 (94.23 MB)

 Trainable params: 24,649,731 (94.03 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [15]:
img_path_1 = 'Volunteers Data\MADAM REHANA\hand (after 15 days).jpg'  # First image path (before)
img_path_2 = 'Volunteers Data\MADAM REHANA\hand (after 35 days).jpg'  # Second image path (after)

img1 = image.load_img(img_path_1, target_size=(224, 224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1, axis=0)
img1 = preprocess_input(img1)

img2 = image.load_img(img_path_2, target_size=(224, 224))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis=0)
img2 = preprocess_input(img2)

# Predict whether the skin quality has improved, deteriorated, or remained the same
prediction = siamese_model.predict([img1, img2])

# Interpret the prediction
class_labels = ['improved', 'no change', 'deteriorated']
predicted_class = class_labels[np.argmax(prediction[0])]
print(f"Prediction: Skin quality has {predicted_class}.")


<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\arham\AppData\Local\Temp\ipykernel_20396\2853168960.py:1: SyntaxWarning: invalid escape sequence '\M'
  img_path_1 = 'Volunteers Data\MADAM REHANA\hand (after 15 days).jpg'  # First image path (before)
C:\Users\arham\AppData\Local\Temp\ipykernel_20396\2853168960.py:2: SyntaxWarning: invalid escape sequence '\M'
  img_path_2 = 'Volunteers Data\MADAM REHANA\hand (after 35 days).jpg'  # Second image path (after)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Skin quality has improved.
